There is a pipeline for task "clean and dirty" on Kaggle 
*   The best accuracy is 0.891
*   augmentation + resnet18 + Bagging

Main steps:

1.   importing libraries
2.   removing background of pictures
3.   augmentation
4.   transforming image to matrix(dim = 2) for BaggingClassifier
5.   defining base_estimator - PytorchModel
5.   fitting/prediction


###IMPORTING

---



In [ ]:
import torch
import torchvision.transforms as transforms
import glob
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import time
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import sklearn



import random
seed = 77
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True


###setup config file for Kaggle 
#####For downloading datasets/sending predictions

In [ ]:
from google.colab import files
files.upload() 

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c platesv2


In [ ]:
!unzip plates.zip 

In [ ]:
import os
!rm -rf train val sample_data test
data_root = '/content/plates'
!rm -rf /content/plates/.DS_Store
!rm -rf /content/plates/train/dirty/.DS_Store
!rm -rf /content/plates/train/cleaned/.DS_Store
print(os.listdir(data_root))

['train', 'test']


#Removing background for all pictures

In [ ]:
def show_input(path, title=''):
    image = Image.open(path)
    plt.imshow(image), plt.axis("off")
    plt.title(title)
    plt.pause(0.001)

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import imageio


def remove_back(listdir):
  for j in listdir:
    image_bgr = cv2.imread(j)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    rectangle = (10, 10, image_bgr.shape[1]-20, image_bgr.shape[0]-20)

    # Create initial mask
    mask = np.zeros(image_rgb.shape[:2], np.uint8)

    # Create temporary arrays
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)

    # Run grabCut
    cv2.grabCut(image_rgb, mask, rectangle, bgdModel, fgdModel, 10, cv2.GC_INIT_WITH_RECT) 

    # Create mask where sure and likely backgrounds set to 0, otherwise 1
    mask_2 = np.where((mask==2) | (mask==0), 0, 1).astype('uint8')

    # Multiply image with new mask to subtract background
    image_rgb_nobg = image_rgb * mask_2[:, :, np.newaxis]
    background = image_rgb - image_rgb_nobg

    # Change all pixels in the background that are not black to white
    background[np.where((background > [0, 0, 0]).all(axis = 2))] = [255, 255, 255]

    #Add the background and the image
    out_img = background + image_rgb_nobg   
    
    imageio.imwrite(j, out_img)

train_list = glob.glob('/content/plates/train/*/*.jpg')
test_list = glob.glob('/content/plates/test/*.jpg')

lists= [train_list, test_list]

for j in lists:
  remove_back(j)

for j in train_list[:10]:
  show_input(j)



##Creating train, test directories 


In [ ]:
import os
import shutil 
from tqdm import tqdm

#Creating
train_dir = 'train'
test_dir = 'test'

class_names = ['cleaned', 'dirty']

for class_name in class_names:
  os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)


#Copying
for class_name in class_names:
  os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)

for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
      dest_dir = os.path.join(train_dir, class_name)  
      shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))  

100%|██████████| 20/20 [00:00<00:00, 6774.84it/s]


'test/unknown'

In [ ]:
import torch.nn.functional as F
class CustomNet(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.model = torchvision.models.resnet18(pretrained=True)
    for param in self.model.parameters():
      param.requires_grad = False

    self.model.fc = torch.nn.Sequential(
    torch.nn.Linear(self.model.fc.in_features, 512),
    torch.nn.BatchNorm1d(512),
    #torch.nn.Dropout(p = 0.7),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.BatchNorm1d(256),
    #torch.nn.Dropout(p = 0.7),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 2))

  def forward(self, x):
    x = self.model(x)
    return x


#BAGGING


In [ ]:
train_transforms = transforms.Compose([
          transforms.RandomPerspective(distortion_scale=0.2, p=0.1, interpolation=3, fill=255),
          transforms.RandomChoice([transforms.CenterCrop(180),
                                  transforms.CenterCrop(160),
                                  #transforms.CenterCrop(140),
                                  #transforms.CenterCrop(120),
                                  transforms.Compose([transforms.CenterCrop(280),transforms.Grayscale(3,),]),
                                  transforms.Compose([transforms.CenterCrop(200),transforms.Grayscale(3,),])]),
          transforms.Resize((224, 224)),
          transforms.ColorJitter(contrast=4),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
      ])
  
val_transforms = transforms.Compose([
          transforms.RandomPerspective(distortion_scale=0.2, p=0.1, interpolation=3, fill=255),
          transforms.RandomChoice([transforms.CenterCrop(180),
                                  transforms.CenterCrop(160),
                                  #transforms.CenterCrop(140),
                                  #transforms.CenterCrop(120),
                                  transforms.Compose([transforms.CenterCrop(280), transforms.Grayscale(3,),]),
                                  transforms.Compose([transforms.CenterCrop(200),transforms.Grayscale(3,),])]),
          transforms.Resize((224, 224)),
          transforms.ColorJitter(contrast=4),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      ])


###Creating custom Dataloader class for getting original tuple + path of image

In [ ]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super().__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    

##Transforming image to matrix(dim = 2) for BaggingClassifier

In [ ]:
import glob
from PIL import Image
def to_tensor(phase = 'train'):
  transforms_in_place = transforms.Compose([
          transforms.Resize((224, 224)),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
  
  if phase == 'train':
    train = []
    images=glob.glob("/content/train/*/*jpg") 
    for image in images:
      img = Image.open(image)
      img = transforms_in_place(img)
      train.append((image.split('/')[3], img))
    X = np.zeros((0,150528))
    y_train = []
    for j in train:
      y_train.append(j[0])
      X = np.concatenate((X,torch.unsqueeze(j[1][:,:,:].reshape(-1), 0)), axis = 0)
    Y = []
    for j in y_train:
      if j == 'dirty':
        Y.append(1)
      else:
        Y.append(0)
    return X, Y
  
  else:
    test = []
    images=glob.glob("/content/test/unknown/*jpg") 
    for image in images:
      img = Image.open(image)
      img = transforms_in_place(img)
      test.append(img)
    X = np.zeros((0,150528))
    for j in test:
      X = np.concatenate((X,torch.unsqueeze(j[:,:,:].reshape(-1), 0)), axis = 0)
    return X

X_train, y_train = to_tensor(phase = 'train')   
X_test = to_tensor(phase = 'test')

X_train.shape, X_test.shape  # N_samples * 3 * 244 * 244

((40, 150528), (744, 150528))

##PytorchModel - base_estimator

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import sklearn
class PytorchModel(sklearn.base.BaseEstimator):
  def __init__(self, net_type, net_params, optim_type, optim_params, loss_fn,
               input_shape, batch_size=20, accuracy_tol=0.02, tol_epochs=10,
               cuda=True):
    self.net_type = net_type
    self.net_params = net_params
    self.optim_type = optim_type
    self.optim_params = optim_params
    self.loss_fn = loss_fn

    self.input_shape = input_shape
    self.batch_size = batch_size
    self.accuracy_tol = accuracy_tol
    self.tol_epochs = tol_epochs
    self.cuda = cuda
    self.scheduler =  torch.optim.lr_scheduler.StepLR

  """FIT FUNC"""
  def fit(self, X, y):
    self.net = self.net_type(**self.net_params)
    trainloss = []
    trainacc = []
    if self.cuda:
      self.net = self.net.cuda()
    self.optim = self.optim_type(self.net.parameters(), **self.optim_params)
    self.schedul = self.scheduler(self.optim, step_size = 5)
    uniq_classes = [0,1]
    self.classes_ = uniq_classes
 
    
    #hack
    path = '/content/train'
    train_dataset = torchvision.datasets.ImageFolder(path, transform = train_transforms)
    train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, drop_last=False)
    #hack
    
    last_accuracies = []
    epoch = 0
    keep_training = True
    while keep_training:
      self.net.train()
      train_samples_count = 0
      true_train_samples_count = 0

      self.schedul.step()
      for batch in train_loader:
        x_data, y_data = batch[0], batch[1]
        if self.cuda:
          x_data = x_data.cuda()
          y_data = y_data.cuda()

        y_pred = self.net(x_data)
        loss = self.loss_fn(y_pred, y_data)

        self.optim.zero_grad()
        loss.backward()
        self.optim.step()

        y_pred = y_pred.argmax(dim=1, keepdim=False)
        true_classified = (y_pred == y_data).sum().item()
        true_train_samples_count += true_classified
        train_samples_count += len(x_data)

      train_accuracy = true_train_samples_count / train_samples_count
      last_accuracies.append(train_accuracy)
      print('metrics for epoch', 'loss:',loss, 'acc:',train_accuracy)
      trainloss.append(loss)
      trainacc.append(train_accuracy)
      
      if len(last_accuracies) > self.tol_epochs:
        last_accuracies.pop(0)

      if len(last_accuracies) == self.tol_epochs:
        accuracy_difference = max(last_accuracies) - min(last_accuracies)
        if accuracy_difference <= self.accuracy_tol:
          keep_training = False

  
  """PREDICT FUNC"""
  def predict_proba(self, X, y=None):

    #hack
    path = '/content/test'
    test_dataset = ImageFolderWithPaths(path, val_transforms)
    test_loader = DataLoader(test_dataset, batch_size=50,shuffle=False, drop_last=False)
    #hack
    

    self.net.eval()
    predictions = []
    im_path = []
    with torch.no_grad():
      for batch in test_loader:
        x_data, y_data, paths = batch[0], batch[1], batch[2]
        if self.cuda:
          x_data = x_data.cuda()
          y_data = y_data.cuda()
        im_path.extend(paths)
        y_pred = self.net(x_data)

        predictions.append(y_pred.detach().cpu().numpy())
    #print(im_path)
    predictions = np.concatenate(predictions)
    return predictions

  def predict(self, X, y=None):
    predictions = self.predict_proba(X, y)
    predictions = predictions.argmax(axis=1)
    return predictions

#BaggingClassifier

In [ ]:
base_model = PytorchModel(net_type=CustomNet, net_params=dict(), optim_type=torch.optim.Adam,
                          optim_params={"lr": 1e-3}, loss_fn=torch.nn.CrossEntropyLoss(),
                          input_shape=(3, 224, 224), batch_size=10, accuracy_tol=0.1,
                          tol_epochs=7, cuda=True)

from sklearn.ensemble import BaggingClassifier
meta_classifier = BaggingClassifier(base_estimator=base_model, n_estimators=25) 
meta_classifier.fit(X_train, y_train) 



#PEDICTION

In [ ]:
test_img_paths = ['/content/test/unknown/0000.jpg', '/content/test/unknown/0001.jpg', '/content/test/unknown/0002.jpg', '/content/test/unknown/0003.jpg', '/content/test/unknown/0004.jpg', '/content/test/unknown/0005.jpg', '/content/test/unknown/0006.jpg', '/content/test/unknown/0007.jpg', '/content/test/unknown/0008.jpg', '/content/test/unknown/0009.jpg', '/content/test/unknown/0010.jpg', '/content/test/unknown/0011.jpg', '/content/test/unknown/0012.jpg', '/content/test/unknown/0013.jpg', '/content/test/unknown/0014.jpg', '/content/test/unknown/0015.jpg', '/content/test/unknown/0016.jpg', '/content/test/unknown/0017.jpg', '/content/test/unknown/0018.jpg', '/content/test/unknown/0019.jpg', '/content/test/unknown/0020.jpg', '/content/test/unknown/0021.jpg', '/content/test/unknown/0022.jpg', '/content/test/unknown/0023.jpg', '/content/test/unknown/0024.jpg', '/content/test/unknown/0025.jpg', '/content/test/unknown/0026.jpg', '/content/test/unknown/0027.jpg', '/content/test/unknown/0028.jpg', '/content/test/unknown/0029.jpg', '/content/test/unknown/0030.jpg', '/content/test/unknown/0031.jpg', '/content/test/unknown/0032.jpg', '/content/test/unknown/0033.jpg', '/content/test/unknown/0034.jpg', '/content/test/unknown/0035.jpg', '/content/test/unknown/0036.jpg', '/content/test/unknown/0037.jpg', '/content/test/unknown/0038.jpg', '/content/test/unknown/0039.jpg', '/content/test/unknown/0040.jpg', '/content/test/unknown/0041.jpg', '/content/test/unknown/0042.jpg', '/content/test/unknown/0043.jpg', '/content/test/unknown/0044.jpg', '/content/test/unknown/0045.jpg', '/content/test/unknown/0046.jpg', '/content/test/unknown/0047.jpg', '/content/test/unknown/0048.jpg', '/content/test/unknown/0049.jpg', '/content/test/unknown/0050.jpg', '/content/test/unknown/0051.jpg', '/content/test/unknown/0052.jpg', '/content/test/unknown/0053.jpg', '/content/test/unknown/0054.jpg', '/content/test/unknown/0055.jpg', '/content/test/unknown/0056.jpg', '/content/test/unknown/0057.jpg', '/content/test/unknown/0058.jpg', '/content/test/unknown/0059.jpg', '/content/test/unknown/0060.jpg', '/content/test/unknown/0061.jpg', '/content/test/unknown/0062.jpg', '/content/test/unknown/0063.jpg', '/content/test/unknown/0064.jpg', '/content/test/unknown/0065.jpg', '/content/test/unknown/0066.jpg', '/content/test/unknown/0067.jpg', '/content/test/unknown/0068.jpg', '/content/test/unknown/0069.jpg', '/content/test/unknown/0070.jpg', '/content/test/unknown/0071.jpg', '/content/test/unknown/0072.jpg', '/content/test/unknown/0073.jpg', '/content/test/unknown/0074.jpg', '/content/test/unknown/0075.jpg', '/content/test/unknown/0076.jpg', '/content/test/unknown/0077.jpg', '/content/test/unknown/0078.jpg', '/content/test/unknown/0079.jpg', '/content/test/unknown/0080.jpg', '/content/test/unknown/0081.jpg', '/content/test/unknown/0082.jpg', '/content/test/unknown/0083.jpg', '/content/test/unknown/0084.jpg', '/content/test/unknown/0085.jpg', '/content/test/unknown/0086.jpg', '/content/test/unknown/0087.jpg', '/content/test/unknown/0088.jpg', '/content/test/unknown/0089.jpg', '/content/test/unknown/0090.jpg', '/content/test/unknown/0091.jpg', '/content/test/unknown/0092.jpg', '/content/test/unknown/0093.jpg', '/content/test/unknown/0094.jpg', '/content/test/unknown/0095.jpg', '/content/test/unknown/0096.jpg', '/content/test/unknown/0097.jpg', '/content/test/unknown/0098.jpg', '/content/test/unknown/0099.jpg', '/content/test/unknown/0100.jpg', '/content/test/unknown/0101.jpg', '/content/test/unknown/0102.jpg', '/content/test/unknown/0103.jpg', '/content/test/unknown/0104.jpg', '/content/test/unknown/0105.jpg', '/content/test/unknown/0106.jpg', '/content/test/unknown/0107.jpg', '/content/test/unknown/0108.jpg', '/content/test/unknown/0109.jpg', '/content/test/unknown/0110.jpg', '/content/test/unknown/0111.jpg', '/content/test/unknown/0112.jpg', '/content/test/unknown/0113.jpg', '/content/test/unknown/0114.jpg', '/content/test/unknown/0115.jpg', '/content/test/unknown/0116.jpg', '/content/test/unknown/0117.jpg', '/content/test/unknown/0118.jpg', '/content/test/unknown/0119.jpg', '/content/test/unknown/0120.jpg', '/content/test/unknown/0121.jpg', '/content/test/unknown/0122.jpg', '/content/test/unknown/0123.jpg', '/content/test/unknown/0124.jpg', '/content/test/unknown/0125.jpg', '/content/test/unknown/0126.jpg', '/content/test/unknown/0127.jpg', '/content/test/unknown/0128.jpg', '/content/test/unknown/0129.jpg', '/content/test/unknown/0130.jpg', '/content/test/unknown/0131.jpg', '/content/test/unknown/0132.jpg', '/content/test/unknown/0133.jpg', '/content/test/unknown/0134.jpg', '/content/test/unknown/0135.jpg', '/content/test/unknown/0136.jpg', '/content/test/unknown/0137.jpg', '/content/test/unknown/0138.jpg', '/content/test/unknown/0139.jpg', '/content/test/unknown/0140.jpg', '/content/test/unknown/0141.jpg', '/content/test/unknown/0142.jpg', '/content/test/unknown/0143.jpg', '/content/test/unknown/0144.jpg', '/content/test/unknown/0145.jpg', '/content/test/unknown/0146.jpg', '/content/test/unknown/0147.jpg', '/content/test/unknown/0148.jpg', '/content/test/unknown/0149.jpg', '/content/test/unknown/0150.jpg', '/content/test/unknown/0151.jpg', '/content/test/unknown/0152.jpg', '/content/test/unknown/0153.jpg', '/content/test/unknown/0154.jpg', '/content/test/unknown/0155.jpg', '/content/test/unknown/0156.jpg', '/content/test/unknown/0157.jpg', '/content/test/unknown/0158.jpg', '/content/test/unknown/0159.jpg', '/content/test/unknown/0160.jpg', '/content/test/unknown/0161.jpg', '/content/test/unknown/0162.jpg', '/content/test/unknown/0163.jpg', '/content/test/unknown/0164.jpg', '/content/test/unknown/0165.jpg', '/content/test/unknown/0166.jpg', '/content/test/unknown/0167.jpg', '/content/test/unknown/0168.jpg', '/content/test/unknown/0169.jpg', '/content/test/unknown/0170.jpg', '/content/test/unknown/0171.jpg', '/content/test/unknown/0172.jpg', '/content/test/unknown/0173.jpg', '/content/test/unknown/0174.jpg', '/content/test/unknown/0175.jpg', '/content/test/unknown/0176.jpg', '/content/test/unknown/0177.jpg', '/content/test/unknown/0178.jpg', '/content/test/unknown/0179.jpg', '/content/test/unknown/0180.jpg', '/content/test/unknown/0181.jpg', '/content/test/unknown/0182.jpg', '/content/test/unknown/0183.jpg', '/content/test/unknown/0184.jpg', '/content/test/unknown/0185.jpg', '/content/test/unknown/0186.jpg', '/content/test/unknown/0187.jpg', '/content/test/unknown/0188.jpg', '/content/test/unknown/0189.jpg', '/content/test/unknown/0190.jpg', '/content/test/unknown/0191.jpg', '/content/test/unknown/0192.jpg', '/content/test/unknown/0193.jpg', '/content/test/unknown/0194.jpg', '/content/test/unknown/0195.jpg', '/content/test/unknown/0196.jpg', '/content/test/unknown/0197.jpg', '/content/test/unknown/0198.jpg', '/content/test/unknown/0199.jpg', '/content/test/unknown/0200.jpg', '/content/test/unknown/0201.jpg', '/content/test/unknown/0202.jpg', '/content/test/unknown/0203.jpg', '/content/test/unknown/0204.jpg', '/content/test/unknown/0205.jpg', '/content/test/unknown/0206.jpg', '/content/test/unknown/0207.jpg', '/content/test/unknown/0208.jpg', '/content/test/unknown/0209.jpg', '/content/test/unknown/0210.jpg', '/content/test/unknown/0211.jpg', '/content/test/unknown/0212.jpg', '/content/test/unknown/0213.jpg', '/content/test/unknown/0214.jpg', '/content/test/unknown/0215.jpg', '/content/test/unknown/0216.jpg', '/content/test/unknown/0217.jpg', '/content/test/unknown/0218.jpg', '/content/test/unknown/0219.jpg', '/content/test/unknown/0220.jpg', '/content/test/unknown/0221.jpg', '/content/test/unknown/0222.jpg', '/content/test/unknown/0223.jpg', '/content/test/unknown/0224.jpg', '/content/test/unknown/0225.jpg', '/content/test/unknown/0226.jpg', '/content/test/unknown/0227.jpg', '/content/test/unknown/0228.jpg', '/content/test/unknown/0229.jpg', '/content/test/unknown/0230.jpg', '/content/test/unknown/0231.jpg', '/content/test/unknown/0232.jpg', '/content/test/unknown/0233.jpg', '/content/test/unknown/0234.jpg', '/content/test/unknown/0235.jpg', '/content/test/unknown/0236.jpg', '/content/test/unknown/0237.jpg', '/content/test/unknown/0238.jpg', '/content/test/unknown/0239.jpg', '/content/test/unknown/0240.jpg', '/content/test/unknown/0241.jpg', '/content/test/unknown/0242.jpg', '/content/test/unknown/0243.jpg', '/content/test/unknown/0244.jpg', '/content/test/unknown/0245.jpg', '/content/test/unknown/0246.jpg', '/content/test/unknown/0247.jpg', '/content/test/unknown/0248.jpg', '/content/test/unknown/0249.jpg', '/content/test/unknown/0250.jpg', '/content/test/unknown/0251.jpg', '/content/test/unknown/0252.jpg', '/content/test/unknown/0253.jpg', '/content/test/unknown/0254.jpg', '/content/test/unknown/0255.jpg', '/content/test/unknown/0256.jpg', '/content/test/unknown/0257.jpg', '/content/test/unknown/0258.jpg', '/content/test/unknown/0259.jpg', '/content/test/unknown/0260.jpg', '/content/test/unknown/0261.jpg', '/content/test/unknown/0262.jpg', '/content/test/unknown/0263.jpg', '/content/test/unknown/0264.jpg', '/content/test/unknown/0265.jpg', '/content/test/unknown/0266.jpg', '/content/test/unknown/0267.jpg', '/content/test/unknown/0268.jpg', '/content/test/unknown/0269.jpg', '/content/test/unknown/0270.jpg', '/content/test/unknown/0271.jpg', '/content/test/unknown/0272.jpg', '/content/test/unknown/0273.jpg', '/content/test/unknown/0274.jpg', '/content/test/unknown/0275.jpg', '/content/test/unknown/0276.jpg', '/content/test/unknown/0277.jpg', '/content/test/unknown/0278.jpg', '/content/test/unknown/0279.jpg', '/content/test/unknown/0280.jpg', '/content/test/unknown/0281.jpg', '/content/test/unknown/0282.jpg', '/content/test/unknown/0283.jpg', '/content/test/unknown/0284.jpg', '/content/test/unknown/0285.jpg', '/content/test/unknown/0286.jpg', '/content/test/unknown/0287.jpg', '/content/test/unknown/0288.jpg', '/content/test/unknown/0289.jpg', '/content/test/unknown/0290.jpg', '/content/test/unknown/0291.jpg', '/content/test/unknown/0292.jpg', '/content/test/unknown/0293.jpg', '/content/test/unknown/0294.jpg', '/content/test/unknown/0295.jpg', '/content/test/unknown/0296.jpg', '/content/test/unknown/0297.jpg', '/content/test/unknown/0298.jpg', '/content/test/unknown/0299.jpg', '/content/test/unknown/0300.jpg', '/content/test/unknown/0301.jpg', '/content/test/unknown/0302.jpg', '/content/test/unknown/0303.jpg', '/content/test/unknown/0304.jpg', '/content/test/unknown/0305.jpg', '/content/test/unknown/0306.jpg', '/content/test/unknown/0307.jpg', '/content/test/unknown/0308.jpg', '/content/test/unknown/0309.jpg', '/content/test/unknown/0310.jpg', '/content/test/unknown/0311.jpg', '/content/test/unknown/0312.jpg', '/content/test/unknown/0313.jpg', '/content/test/unknown/0314.jpg', '/content/test/unknown/0315.jpg', '/content/test/unknown/0316.jpg', '/content/test/unknown/0317.jpg', '/content/test/unknown/0318.jpg', '/content/test/unknown/0319.jpg', '/content/test/unknown/0320.jpg', '/content/test/unknown/0321.jpg', '/content/test/unknown/0322.jpg', '/content/test/unknown/0323.jpg', '/content/test/unknown/0324.jpg', '/content/test/unknown/0325.jpg', '/content/test/unknown/0326.jpg', '/content/test/unknown/0327.jpg', '/content/test/unknown/0328.jpg', '/content/test/unknown/0329.jpg', '/content/test/unknown/0330.jpg', '/content/test/unknown/0331.jpg', '/content/test/unknown/0332.jpg', '/content/test/unknown/0333.jpg', '/content/test/unknown/0334.jpg', '/content/test/unknown/0335.jpg', '/content/test/unknown/0336.jpg', '/content/test/unknown/0337.jpg', '/content/test/unknown/0338.jpg', '/content/test/unknown/0339.jpg', '/content/test/unknown/0340.jpg', '/content/test/unknown/0341.jpg', '/content/test/unknown/0342.jpg', '/content/test/unknown/0343.jpg', '/content/test/unknown/0344.jpg', '/content/test/unknown/0345.jpg', '/content/test/unknown/0346.jpg', '/content/test/unknown/0347.jpg', '/content/test/unknown/0348.jpg', '/content/test/unknown/0349.jpg', '/content/test/unknown/0350.jpg', '/content/test/unknown/0351.jpg', '/content/test/unknown/0352.jpg', '/content/test/unknown/0353.jpg', '/content/test/unknown/0354.jpg', '/content/test/unknown/0355.jpg', '/content/test/unknown/0356.jpg', '/content/test/unknown/0357.jpg', '/content/test/unknown/0358.jpg', '/content/test/unknown/0359.jpg', '/content/test/unknown/0360.jpg', '/content/test/unknown/0361.jpg', '/content/test/unknown/0362.jpg', '/content/test/unknown/0363.jpg', '/content/test/unknown/0364.jpg', '/content/test/unknown/0365.jpg', '/content/test/unknown/0366.jpg', '/content/test/unknown/0367.jpg', '/content/test/unknown/0368.jpg', '/content/test/unknown/0369.jpg', '/content/test/unknown/0370.jpg', '/content/test/unknown/0371.jpg', '/content/test/unknown/0372.jpg', '/content/test/unknown/0373.jpg', '/content/test/unknown/0374.jpg', '/content/test/unknown/0375.jpg', '/content/test/unknown/0376.jpg', '/content/test/unknown/0377.jpg', '/content/test/unknown/0378.jpg', '/content/test/unknown/0379.jpg', '/content/test/unknown/0380.jpg', '/content/test/unknown/0381.jpg', '/content/test/unknown/0382.jpg', '/content/test/unknown/0383.jpg', '/content/test/unknown/0384.jpg', '/content/test/unknown/0385.jpg', '/content/test/unknown/0386.jpg', '/content/test/unknown/0387.jpg', '/content/test/unknown/0388.jpg', '/content/test/unknown/0389.jpg', '/content/test/unknown/0390.jpg', '/content/test/unknown/0391.jpg', '/content/test/unknown/0392.jpg', '/content/test/unknown/0393.jpg', '/content/test/unknown/0394.jpg', '/content/test/unknown/0395.jpg', '/content/test/unknown/0396.jpg', '/content/test/unknown/0397.jpg', '/content/test/unknown/0398.jpg', '/content/test/unknown/0399.jpg', '/content/test/unknown/0400.jpg', '/content/test/unknown/0401.jpg', '/content/test/unknown/0402.jpg', '/content/test/unknown/0403.jpg', '/content/test/unknown/0404.jpg', '/content/test/unknown/0405.jpg', '/content/test/unknown/0406.jpg', '/content/test/unknown/0407.jpg', '/content/test/unknown/0408.jpg', '/content/test/unknown/0409.jpg', '/content/test/unknown/0410.jpg', '/content/test/unknown/0411.jpg', '/content/test/unknown/0412.jpg', '/content/test/unknown/0413.jpg', '/content/test/unknown/0414.jpg', '/content/test/unknown/0415.jpg', '/content/test/unknown/0416.jpg', '/content/test/unknown/0417.jpg', '/content/test/unknown/0418.jpg', '/content/test/unknown/0419.jpg', '/content/test/unknown/0420.jpg', '/content/test/unknown/0421.jpg', '/content/test/unknown/0422.jpg', '/content/test/unknown/0423.jpg', '/content/test/unknown/0424.jpg', '/content/test/unknown/0425.jpg', '/content/test/unknown/0426.jpg', '/content/test/unknown/0427.jpg', '/content/test/unknown/0428.jpg', '/content/test/unknown/0429.jpg', '/content/test/unknown/0430.jpg', '/content/test/unknown/0431.jpg', '/content/test/unknown/0432.jpg', '/content/test/unknown/0433.jpg', '/content/test/unknown/0434.jpg', '/content/test/unknown/0435.jpg', '/content/test/unknown/0436.jpg', '/content/test/unknown/0437.jpg', '/content/test/unknown/0438.jpg', '/content/test/unknown/0439.jpg', '/content/test/unknown/0440.jpg', '/content/test/unknown/0441.jpg', '/content/test/unknown/0442.jpg', '/content/test/unknown/0443.jpg', '/content/test/unknown/0444.jpg', '/content/test/unknown/0445.jpg', '/content/test/unknown/0446.jpg', '/content/test/unknown/0447.jpg', '/content/test/unknown/0448.jpg', '/content/test/unknown/0449.jpg', '/content/test/unknown/0450.jpg', '/content/test/unknown/0451.jpg', '/content/test/unknown/0452.jpg', '/content/test/unknown/0453.jpg', '/content/test/unknown/0454.jpg', '/content/test/unknown/0455.jpg', '/content/test/unknown/0456.jpg', '/content/test/unknown/0457.jpg', '/content/test/unknown/0458.jpg', '/content/test/unknown/0459.jpg', '/content/test/unknown/0460.jpg', '/content/test/unknown/0461.jpg', '/content/test/unknown/0462.jpg', '/content/test/unknown/0463.jpg', '/content/test/unknown/0464.jpg', '/content/test/unknown/0465.jpg', '/content/test/unknown/0466.jpg', '/content/test/unknown/0467.jpg', '/content/test/unknown/0468.jpg', '/content/test/unknown/0469.jpg', '/content/test/unknown/0470.jpg', '/content/test/unknown/0471.jpg', '/content/test/unknown/0472.jpg', '/content/test/unknown/0473.jpg', '/content/test/unknown/0474.jpg', '/content/test/unknown/0475.jpg', '/content/test/unknown/0476.jpg', '/content/test/unknown/0477.jpg', '/content/test/unknown/0478.jpg', '/content/test/unknown/0479.jpg', '/content/test/unknown/0480.jpg', '/content/test/unknown/0481.jpg', '/content/test/unknown/0482.jpg', '/content/test/unknown/0483.jpg', '/content/test/unknown/0484.jpg', '/content/test/unknown/0485.jpg', '/content/test/unknown/0486.jpg', '/content/test/unknown/0487.jpg', '/content/test/unknown/0488.jpg', '/content/test/unknown/0489.jpg', '/content/test/unknown/0490.jpg', '/content/test/unknown/0491.jpg', '/content/test/unknown/0492.jpg', '/content/test/unknown/0493.jpg', '/content/test/unknown/0494.jpg', '/content/test/unknown/0495.jpg', '/content/test/unknown/0496.jpg', '/content/test/unknown/0497.jpg', '/content/test/unknown/0498.jpg', '/content/test/unknown/0499.jpg', '/content/test/unknown/0500.jpg', '/content/test/unknown/0501.jpg', '/content/test/unknown/0502.jpg', '/content/test/unknown/0503.jpg', '/content/test/unknown/0504.jpg', '/content/test/unknown/0505.jpg', '/content/test/unknown/0506.jpg', '/content/test/unknown/0507.jpg', '/content/test/unknown/0508.jpg', '/content/test/unknown/0509.jpg', '/content/test/unknown/0510.jpg', '/content/test/unknown/0511.jpg', '/content/test/unknown/0512.jpg', '/content/test/unknown/0513.jpg', '/content/test/unknown/0514.jpg', '/content/test/unknown/0515.jpg', '/content/test/unknown/0516.jpg', '/content/test/unknown/0517.jpg', '/content/test/unknown/0518.jpg', '/content/test/unknown/0519.jpg', '/content/test/unknown/0520.jpg', '/content/test/unknown/0521.jpg', '/content/test/unknown/0522.jpg', '/content/test/unknown/0523.jpg', '/content/test/unknown/0524.jpg', '/content/test/unknown/0525.jpg', '/content/test/unknown/0526.jpg', '/content/test/unknown/0527.jpg', '/content/test/unknown/0528.jpg', '/content/test/unknown/0529.jpg', '/content/test/unknown/0530.jpg', '/content/test/unknown/0531.jpg', '/content/test/unknown/0532.jpg', '/content/test/unknown/0533.jpg', '/content/test/unknown/0534.jpg', '/content/test/unknown/0535.jpg', '/content/test/unknown/0536.jpg', '/content/test/unknown/0537.jpg', '/content/test/unknown/0538.jpg', '/content/test/unknown/0539.jpg', '/content/test/unknown/0540.jpg', '/content/test/unknown/0541.jpg', '/content/test/unknown/0542.jpg', '/content/test/unknown/0543.jpg', '/content/test/unknown/0544.jpg', '/content/test/unknown/0545.jpg', '/content/test/unknown/0546.jpg', '/content/test/unknown/0547.jpg', '/content/test/unknown/0548.jpg', '/content/test/unknown/0549.jpg', '/content/test/unknown/0550.jpg', '/content/test/unknown/0551.jpg', '/content/test/unknown/0552.jpg', '/content/test/unknown/0553.jpg', '/content/test/unknown/0554.jpg', '/content/test/unknown/0555.jpg', '/content/test/unknown/0556.jpg', '/content/test/unknown/0557.jpg', '/content/test/unknown/0558.jpg', '/content/test/unknown/0559.jpg', '/content/test/unknown/0560.jpg', '/content/test/unknown/0561.jpg', '/content/test/unknown/0562.jpg', '/content/test/unknown/0563.jpg', '/content/test/unknown/0564.jpg', '/content/test/unknown/0565.jpg', '/content/test/unknown/0566.jpg', '/content/test/unknown/0567.jpg', '/content/test/unknown/0568.jpg', '/content/test/unknown/0569.jpg', '/content/test/unknown/0570.jpg', '/content/test/unknown/0571.jpg', '/content/test/unknown/0572.jpg', '/content/test/unknown/0573.jpg', '/content/test/unknown/0574.jpg', '/content/test/unknown/0575.jpg', '/content/test/unknown/0576.jpg', '/content/test/unknown/0577.jpg', '/content/test/unknown/0578.jpg', '/content/test/unknown/0579.jpg', '/content/test/unknown/0580.jpg', '/content/test/unknown/0581.jpg', '/content/test/unknown/0582.jpg', '/content/test/unknown/0583.jpg', '/content/test/unknown/0584.jpg', '/content/test/unknown/0585.jpg', '/content/test/unknown/0586.jpg', '/content/test/unknown/0587.jpg', '/content/test/unknown/0588.jpg', '/content/test/unknown/0589.jpg', '/content/test/unknown/0590.jpg', '/content/test/unknown/0591.jpg', '/content/test/unknown/0592.jpg', '/content/test/unknown/0593.jpg', '/content/test/unknown/0594.jpg', '/content/test/unknown/0595.jpg', '/content/test/unknown/0596.jpg', '/content/test/unknown/0597.jpg', '/content/test/unknown/0598.jpg', '/content/test/unknown/0599.jpg', '/content/test/unknown/0600.jpg', '/content/test/unknown/0601.jpg', '/content/test/unknown/0602.jpg', '/content/test/unknown/0603.jpg', '/content/test/unknown/0604.jpg', '/content/test/unknown/0605.jpg', '/content/test/unknown/0606.jpg', '/content/test/unknown/0607.jpg', '/content/test/unknown/0608.jpg', '/content/test/unknown/0609.jpg', '/content/test/unknown/0610.jpg', '/content/test/unknown/0611.jpg', '/content/test/unknown/0612.jpg', '/content/test/unknown/0613.jpg', '/content/test/unknown/0614.jpg', '/content/test/unknown/0615.jpg', '/content/test/unknown/0616.jpg', '/content/test/unknown/0617.jpg', '/content/test/unknown/0618.jpg', '/content/test/unknown/0619.jpg', '/content/test/unknown/0620.jpg', '/content/test/unknown/0621.jpg', '/content/test/unknown/0622.jpg', '/content/test/unknown/0623.jpg', '/content/test/unknown/0624.jpg', '/content/test/unknown/0625.jpg', '/content/test/unknown/0626.jpg', '/content/test/unknown/0627.jpg', '/content/test/unknown/0628.jpg', '/content/test/unknown/0629.jpg', '/content/test/unknown/0630.jpg', '/content/test/unknown/0631.jpg', '/content/test/unknown/0632.jpg', '/content/test/unknown/0633.jpg', '/content/test/unknown/0634.jpg', '/content/test/unknown/0635.jpg', '/content/test/unknown/0636.jpg', '/content/test/unknown/0637.jpg', '/content/test/unknown/0638.jpg', '/content/test/unknown/0639.jpg', '/content/test/unknown/0640.jpg', '/content/test/unknown/0641.jpg', '/content/test/unknown/0642.jpg', '/content/test/unknown/0643.jpg', '/content/test/unknown/0644.jpg', '/content/test/unknown/0645.jpg', '/content/test/unknown/0646.jpg', '/content/test/unknown/0647.jpg', '/content/test/unknown/0648.jpg', '/content/test/unknown/0649.jpg', '/content/test/unknown/0650.jpg', '/content/test/unknown/0651.jpg', '/content/test/unknown/0652.jpg', '/content/test/unknown/0653.jpg', '/content/test/unknown/0654.jpg', '/content/test/unknown/0655.jpg', '/content/test/unknown/0656.jpg', '/content/test/unknown/0657.jpg', '/content/test/unknown/0658.jpg', '/content/test/unknown/0659.jpg', '/content/test/unknown/0660.jpg', '/content/test/unknown/0661.jpg', '/content/test/unknown/0662.jpg', '/content/test/unknown/0663.jpg', '/content/test/unknown/0664.jpg', '/content/test/unknown/0665.jpg', '/content/test/unknown/0666.jpg', '/content/test/unknown/0667.jpg', '/content/test/unknown/0668.jpg', '/content/test/unknown/0669.jpg', '/content/test/unknown/0670.jpg', '/content/test/unknown/0671.jpg', '/content/test/unknown/0672.jpg', '/content/test/unknown/0673.jpg', '/content/test/unknown/0674.jpg', '/content/test/unknown/0675.jpg', '/content/test/unknown/0676.jpg', '/content/test/unknown/0677.jpg', '/content/test/unknown/0678.jpg', '/content/test/unknown/0679.jpg', '/content/test/unknown/0680.jpg', '/content/test/unknown/0681.jpg', '/content/test/unknown/0682.jpg', '/content/test/unknown/0683.jpg', '/content/test/unknown/0684.jpg', '/content/test/unknown/0685.jpg', '/content/test/unknown/0686.jpg', '/content/test/unknown/0687.jpg', '/content/test/unknown/0688.jpg', '/content/test/unknown/0689.jpg', '/content/test/unknown/0690.jpg', '/content/test/unknown/0691.jpg', '/content/test/unknown/0692.jpg', '/content/test/unknown/0693.jpg', '/content/test/unknown/0694.jpg', '/content/test/unknown/0695.jpg', '/content/test/unknown/0696.jpg', '/content/test/unknown/0697.jpg', '/content/test/unknown/0698.jpg', '/content/test/unknown/0699.jpg', '/content/test/unknown/0700.jpg', '/content/test/unknown/0701.jpg', '/content/test/unknown/0702.jpg', '/content/test/unknown/0703.jpg', '/content/test/unknown/0704.jpg', '/content/test/unknown/0705.jpg', '/content/test/unknown/0706.jpg', '/content/test/unknown/0707.jpg', '/content/test/unknown/0708.jpg', '/content/test/unknown/0709.jpg', '/content/test/unknown/0710.jpg', '/content/test/unknown/0711.jpg', '/content/test/unknown/0712.jpg', '/content/test/unknown/0713.jpg', '/content/test/unknown/0714.jpg', '/content/test/unknown/0715.jpg', '/content/test/unknown/0716.jpg', '/content/test/unknown/0717.jpg', '/content/test/unknown/0718.jpg', '/content/test/unknown/0719.jpg', '/content/test/unknown/0720.jpg', '/content/test/unknown/0721.jpg', '/content/test/unknown/0722.jpg', '/content/test/unknown/0723.jpg', '/content/test/unknown/0724.jpg', '/content/test/unknown/0725.jpg', '/content/test/unknown/0726.jpg', '/content/test/unknown/0727.jpg', '/content/test/unknown/0728.jpg', '/content/test/unknown/0729.jpg', '/content/test/unknown/0730.jpg', '/content/test/unknown/0731.jpg', '/content/test/unknown/0732.jpg', '/content/test/unknown/0733.jpg', '/content/test/unknown/0734.jpg', '/content/test/unknown/0735.jpg', '/content/test/unknown/0736.jpg', '/content/test/unknown/0737.jpg', '/content/test/unknown/0738.jpg', '/content/test/unknown/0739.jpg', '/content/test/unknown/0740.jpg', '/content/test/unknown/0741.jpg', '/content/test/unknown/0742.jpg', '/content/test/unknown/0743.jpg']
test_predictions = meta_classifier.predict(X_test)

for img, pred in zip(test_img_paths, test_predictions):
   show_input(img, title=pred)


In [ ]:
import pandas as pd
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})

submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if (pred > 0.5) else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('/content/test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=6)

submission_df.to_csv('submission.csv')
!kaggle competitions submit platesv2 -f submission.csv -m "My submission message"

100% 8.46k/8.46k [00:00<00:00, 14.1kB/s]
Successfully submitted to Cleaned vs Dirty V2